# IEA ETP 2020の技術リストを一覧シートにする on R
- IEA ETP 2020（https://www.iea.org/articles/etp-clean-energy-technology-guide ）のソースコードから技術リストの全データ（JSON形式）をコピペしたものを読み込む
- tidyjsonでtibble形式に整形する

In [144]:
library(tidyverse)
library(tidyjson)
library(openxl)

In [145]:
# データ読み込み
trl <- read_json('trl.json',format='json');

In [146]:
# JSONオブジェクトの中身を確認する
trl %>% 
    gather_array %>% 
    gather_object

,document.id,array.index,name,..JSON
,<int>,<int>,<chr>,<named list>
1,1,1,sectors,"Energy transformation, Power"
2,1,1,techs,"Solar , Photovoltaic , Crystalline silicon"
3,1,1,valueChainStep,Generation
4,1,1,ccTags,"Materials , Renewable electricity, Systems integration , Hydrogen , Synthetic fuels"
5,1,1,TRL,9-10
6,1,1,deployment2070,Moderate deployment: 25-50% (total solar PV share in generation mix)
7,1,1,netZeroPriority,High
8,1,1,leadingCountries,"Australia , China , Germany , India , Italy , Japan , Korea , Mexico , Turkey , United States"
9,1,1,description,"Today, the vast majority of PV modules are based on wafer-based crystalline silicon (c-Si). The manufacturing of c-Si modules typically involves growing ingots of silicon, slicing the ingots into wafers to make solar cells, electrically interconnecting the cells, and encapsulating the strings of cells to form a module. Modules currently use silicon in one of two main forms: single- (sc-Si) or multi- (mc-Si) crystalline modules. Current commercial single-crystalline modules have a higher conversion efficiency of around 14 to 20%. Their efficiency is expected to increase up to 25% in the longer term. Multi-crystalline silicon modules have a more disordered atomic structure leading to lower efficiencies, but they are less expensive. Their efficiency is expected to increase up to 21% in the long term."


In [147]:
# sectorsはEnergy transformation > Powerな階層がarray形式になっているので、いったんgatherしてから横方向に展開する
sectors <- trl %>%
    gather_array %>%
    enter_object(sectors) %>%
    gather_array('level') %>%
    append_values_string('sector') %>%
    as_tibble %>% 
    pivot_wider(
        names_prefix='sector_',
        names_from=level,
        values_from=sector);

In [148]:
# Techsも階層がarray形式になっているので、gatherしてから横方向に展開する
techs <- trl %>%
    gather_array %>%
    enter_object(techs) %>%
    gather_array('level') %>%
    append_values_string('techs') %>%
    as_tibble %>% 
    pivot_wider(
        names_prefix='techs_',
        names_from=level,
        values_from=techs);

In [149]:
# ccTagsは、arrayをいったんgatherしてから、arrayとしてnestする
ccTags <- trl %>%
    gather_array %>%
    enter_object(ccTags) %>%
    gather_array %>%
    append_values_string('ccTag') %>%
    as_tibble %>% 
    select(-array.index.2) %>%
    nest(ccTags=ccTag);

Warning message in gather_array(.):
“array.index column name already exists, changing to array.index.2”


In [150]:
# leadingCountriesも、arrayをいったんgatherしてから、arrayとしてnestする
leadingCountries <- trl %>%
    gather_array %>%
    enter_object(leadingCountries) %>%
    gather_array %>%
    append_values_string('leadingCountry') %>%
    as_tibble %>% 
    select(-array.index.2) %>%
    nest(leadingCountries=leadingCountry);

Warning message in gather_array(.):
“array.index column name already exists, changing to array.index.2”


In [152]:
data <- 
    # string形式のオブジェクトをそのままカラムに
    trl %>% 
    gather_array %>%
    spread_values(
        TRL=jstring(TRL),
        valueChainStep=jstring(valueChainStep),
        deployment2070=jstring(deployment2070),
        netZeroPriority=jstring(netZeroPriority),
        description=jstring(description),
        prototypes=jstring(prototypes),
        costTargets=jstring(costTargets),
        deploymentTargets=jstring(deploymentTargets),
        leadingInitiatives=jstring(leadingInitiatives)
    ) %>% 
    as_tibble %>% 
    # 上で整形したarrayのカラムをjoinする
    left_join(sectors) %>%
    left_join(techs) %>%
    left_join(ccTags) %>%
    left_join(leadingCountries) %>%
    # 並べ替え
    select(
        id=array.index,
        starts_with('sector'),
        starts_with('techs'),
        everything(),
        -document.id);

Joining, by = c("document.id", "array.index")

Joining, by = c("document.id", "array.index")

Joining, by = c("document.id", "array.index")

Joining, by = c("document.id", "array.index")



In [153]:
# データ表示
data

id,sector_1,sector_2,techs_1,techs_2,techs_3,techs_4,TRL,valueChainStep,deployment2070,netZeroPriority,description,prototypes,costTargets,deploymentTargets,leadingInitiatives,ccTags,leadingCountries
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<list>,<list>
1,Energy transformation,Power,Solar,Photovoltaic,Crystalline silicon,NA,9-10,Generation,Moderate deployment: 25-50% (total solar PV share in generation mix),High,"Today, the vast majority of PV modules are based on wafer-based crystalline silicon (c-Si). The manufacturing of c-Si modules typically involves growing ingots of silicon, slicing the ingots into wafers to make solar cells, electrically interconnecting the cells, and encapsulating the strings of cells to form a module. Modules currently use silicon in one of two main forms: single- (sc-Si) or multi- (mc-Si) crystalline modules. Current commercial single-crystalline modules have a higher conversion efficiency of around 14 to 20%. Their efficiency is expected to increase up to 25% in the longer term. Multi-crystalline silicon modules have a more disordered atomic structure leading to lower efficiencies, but they are less expensive. Their efficiency is expected to increase up to 21% in the long term.","* Crystalline silicon technology is the dominating technology today, wih a share of around 95% in total PV cell production in 2017",US-DOE SunShot 2030 targets: Residential PV: USD 50/MWh Commercial PV: USD 40/MWh Utility-scale PV: USD 30/MWh,"Overall PV targets: * China: 105 GW for 2020 already exceeded by the end of 2017 * India: 100 GW by 2022, 150 GW by 2027, 190 GW by 2030 * Japan: 64 GW by 2030 * Korea: 30.8 GW by 2030","* Crystalline silicon technology is the dominating technology today, wih a share of around 95% in total PV cell production in 2017","Materials , Renewable electricity, Systems integration , Hydrogen , Synthetic fuels","Australia , China , Germany , India , Italy , Japan , Korea , Mexico , Turkey , United States"
2,Energy transformation,Power,Solar,Photovoltaic,Thin-film PV,NA,8,Generation,Moderate deployment: 25-50% (total solar PV share in generation mix),Moderate,"A thin-film solar cell is a second generation solar cell that is made by depositing one or more thin layers, or thin film (TF) of photovoltaic material on a substrate, such as glass, plastic or metal. Although emerging thin-film solar technologies may be more expensive to make than conventional silicon, their lighter weight and greater resilience can fill niches in the energy sector","* Thin film technology accounted for around 5% of the PV cell production in 2017. * Germany is a world leader, Bosch CISTech and Oxford PV, a spinoff from Oxford University, are key initiatives. * The U.S. Office of Naval Research funds NREL’s newest research into next generation thin film PV, to reduce reliance on batteries. * Thin film cells on vehicles are being explored by Toyota for small power applications.",Cost: Under 0.2 USD/Wp by 2030 (Solliance),Deployment: None specific to thin-film,"* Thin film technology accounted for around 5% of the PV cell production in 2017. * Germany is a world leader, Bosch CISTech and Oxford PV, a spinoff from Oxford University, are key initiatives. * The U.S. Office of Naval Research funds NREL’s newest research into next generation thin film PV, to reduce reliance on batteries. * Thin film cells on vehicles are being explored by Toyota for small power applications.","Materials , Renewable electricity, Systems integration , Hydrogen , Synthetic fuels","Germany , Japan , United States"
3,Energy transformation,Power,Solar,Photovoltaic,Concentrated PV,NA,9-10,Generation,Moderate deployment: 25-50% (total solar PV share in generation mix),Moderate,Concentrated PV (CPV) technologies use an optical concentrator system which focuses solar radiation onto a small high-efficiency cell. CPV modules can achieve efficiencies of above 40%,"Cumulative installations (already grid-c